### load required packages

In [ ]:
import numpy as np
import pandas as pd

import statsmodels.api as sm
from scipy import optimize
from scipy.optimize import minimize

import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
from getPriceIndex_DCS_t import *
from getPriceIndex import *

### load data

In [ ]:
residual_df = pd.read_csv('1020_withouttime.csv', delimiter=',')

residual_df = residual_df.rename(columns={'transaction_date':  'date'})
# residual_df = residual_df.dropna()

residuals = residual_df[['date', 'Heckman_result_withouttime']]

# date 
list_of_date = np.array(pd.to_datetime(residuals['date']), dtype='datetime64')
T = list_of_date.astype('float')
ols_result = residuals.iloc[:, 1]
discR = np.array(1, dtype='float')

### Kalman filtering

In [ ]:
heckman = Kalman_filtering(list_of_date=list_of_date, ols_resid=residual_df['Heckman_result_withouttime'], discR=discR, phi=1)
actualDate = heckman.DataPreparation()
heckman.ParameterEstimate()

heckman_huber01 = Kalman_filtering(list_of_date=list_of_date, ols_resid=residual_df['Huber_Heckman_result_withouttime01'], discR=discR, phi=1)
heckman_huber01.DataPreparation()
heckman_huber01.ParameterEstimate()

heckman_huber001 = Kalman_filtering(list_of_date=list_of_date, ols_resid=residual_df['Huber_Heckman_result_withouttime001'], discR=discR, phi=1)
heckman_huber001.DataPreparation()
heckman_huber001.ParameterEstimate()

# 0.05
heckman_huber = Kalman_filtering(list_of_date=list_of_date, ols_resid=residual_df['Huber_Heckman_result_withouttime005'], discR=discR, phi=1)
heckman_huber.DataPreparation()
heckman_huber.ParameterEstimate()

In [ ]:

h_opt = minimize(heckman.ComputeLikelihood, 
                      0.5, 
                      method='Nelder-Mead',
                      tol=1e-10)

hh_opt01 = minimize(heckman_huber01.ComputeLikelihood, 
                      0.5, 
                      method='Nelder-Mead',
                      tol=1e-10)

hh_opt001 = minimize(heckman_huber001.ComputeLikelihood, 
                      0.5, 
                      method='Nelder-Mead',
                      tol=1e-10)
# # 0.05
hh_opt = minimize(heckman_huber.ComputeLikelihood, 
                      0.5, 
                      method='Nelder-Mead',
                      tol=1e-10)

# lasso_opt = minimize(lasso.ComputeLikelihood, 
#                       0.5, 
#                       method='Nelder-Mead',
#                       tol=1e-10)


In [ ]:
h_beta = heckman.UpdateBeta(h_opt.x)
h_index=np.exp(h_beta) / np.exp(h_beta[1]) * 100

hh_beta01 = heckman_huber01.UpdateBeta(hh_opt01.x)
hh_index01=np.exp(hh_beta01) / np.exp(hh_beta01[1]) * 100

hh_beta001 = heckman_huber001.UpdateBeta(hh_opt001.x)
hh_index001=np.exp(hh_beta001) / np.exp(hh_beta001[1]) * 100

# 0.05
hh_beta = heckman_huber.UpdateBeta(hh_opt.x)
hh_indexP=np.exp(hh_beta) / np.exp(hh_beta[1]) * 100

dates = np.array(pd.to_datetime(actualDate), dtype='datetime64')
dates = np.datetime_as_string(dates, unit='D')


#### plotting

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": False}]])

fig['layout'].update(height=800, width=1200,
                    title='',
                    showlegend=False,
                    font=dict(family='Times New Roman', size=12))

fig.add_trace(go.Scatter(name='KF with Hackman',
    y=h_index,
    x=dates,
    line=dict(color='blue', dash='dot', width=1)  
    ), secondary_y=False)

fig.add_trace(go.Scatter(name='hh_index01',
    y=hh_index01,
    x=dates,
    line=dict(color='green', dash='dash', width=1)  
    ), secondary_y=False)

fig.add_trace(go.Scatter(name='hh_index001',
    y=hh_index001,
    x=dates,
    line=dict(color='red', dash='solid', width=1)  
    ), secondary_y=False)

fig.add_trace(go.Scatter(name='hh_index005',
    y=hh_indexP,
    x=dates,
    line=dict(color='orange', dash='dashdot', width=1)  
    ), secondary_y=False)

fig['layout']['xaxis'].update(title='Date')
fig['layout']['yaxis'].update(title='Price index')

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', 
                 mirror=True,
                 tickformat="%b\n%Y", 
                 showgrid=False,
                 dtick = 'M1')
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', 
                 mirror=True, 
                 showgrid=False)


fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                   'paper_bgcolor': 'rgba(0,0,0,0)'},
                  font_color='black',
                  bargap=0.1)

fig.show()
fig.write_image('./kf_indices.pdf')

### DCS-t

In [ ]:
run_code = DCSt_filtering(list_of_date=list_of_date, ols_resid=ols_result, discR=discR, phi=1)
actualDate = run_code.DataPreparation()

parameter_estimate = run_code.ParameterEstimate()

In [ ]:
bnds = [(18, None), (None, 1), (3.5, 5)]

opt = minimize(run_code.ComputeLikelihood, 
               [150, 0, 4],
               method='Nelder-Mead',
               bounds=bnds,
               tol=1e-10)

In [ ]:
opt_x = opt.x
final_Beta = run_code.UpdateBeta(opt_x)
dcs_index= np.exp(final_Beta) /np.exp(final_Beta[1]) * 100
dates = np.array(pd.to_datetime(actualDate),dtype='datetime64')
dates = np.datetime_as_string(dates, unit='D')

### infinity nu
opt_x = [1.8000000e+01, 3.04926966e-03, 9999999999999]
final_Beta_L = run_code.UpdateBeta(opt_x)
dcs_index_L= np.exp(final_Beta_L) /np.exp(final_Beta_L[1]) * 100

#### plotting

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": False}]])

fig['layout'].update(height=800, width=1200,
                    title='',
                    showlegend=False,
                    font=dict(family='Times New Roman', size=12))

fig.add_trace(go.Scatter(name='hh_index001',
    y=hh_index001,
    x=dates,
    line=dict(color='blue', dash='dashdot', width=1)  
    ), secondary_y=False)

fig.add_trace(go.Scatter(name='hh_index005',
    y=hh_indexP,
    x=dates,
    line=dict(color='orange', dash='dot', width=1)  
    ), secondary_y=False)

fig.add_trace(go.Scatter(name='DCS-t',
    y=dcs_index,
    x=dates,
    line=dict(color='red', dash='solid', width=1)  
    ), secondary_y=False)

fig.add_trace(go.Scatter(name='DCS-t with large &#957;',
    y=dcs_index_L,
    x=dates,
    line=dict(color='green', dash='dash', width=1)  
    ), secondary_y=False)


fig['layout']['xaxis'].update(title='Date')
fig['layout']['yaxis'].update(title='Price index')

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', 
                 mirror=True,
                 tickformat="%b\n%Y", 
                 showgrid=False,
                 dtick = 'M1')
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', 
                 mirror=True, 
                 showgrid=False)


fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                   'paper_bgcolor': 'rgba(0,0,0,0)'},
                  font_color='black',
                  bargap=0.1)

fig.show()
# fig.write_image('./kf_dcs.pdf')




### compare with other markets

#### load data

In [ ]:
dai = pd.read_csv('index_df.csv')

art_df = pd.read_csv('other_art.csv', delimiter=';')
art_df = art_df.rename(columns={'Unnamed: 0': 'date'})
art_df['date'] = pd.to_datetime(art_df['date'])

art_df = art_df.set_index('date')

sum_all = art_df.sum(axis=1)

In [ ]:
art_df = art_df.loc['2019-03-21':'2022-03-31']
art_df = art_df.apply(lambda x: x.div(art_df.sum(axis=1)))

art_df['date'] =art_df.index
melted_art_df = art_df.melt(id_vars=['date'])
melted_art_df['color'] =''
color1 = 'rgba(30, 56, 136,0.5)' 
color2 = 'rgba(71, 168, 189, 0.5)'
color3 =  'rgba(245, 230, 99, 0.5)'
color4 = 'rgba(255, 173, 105,0.5)'

melted_art_df['color'].loc[melted_art_df['variable'] == 'Post War'] = color1
melted_art_df['color'].loc[melted_art_df['variable'] == 'Contemporary'] = color2
melted_art_df['color'].loc[melted_art_df['variable'] == 'Ultra Contemporary'] = color3
melted_art_df['color'].loc[melted_art_df['variable'] == 'NFT'] = color4

#### plotting

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig['layout'].update(height=800, width=1200,
                    title='',
                    showlegend=False,
                    font=dict(family='Times New Roman', size=12))

fig.add_trace(go.Scatter(y=dai['dcs_index'],
                         x=dai['date'],
                         line=dict(color='rgb(0, 0, 0)', dash='solid', width=2.5)  
                         ), secondary_y=True)

fig.add_trace(go.Bar(x=melted_art_df['date'],
                     y=melted_art_df['value']*100,
                     marker=dict(color=melted_art_df['color'])
                     ), secondary_y=False)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', 
                 mirror=True,
                 tickformat="%b\n%Y", 
                 showgrid=False,
                 dtick = 'M1')
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', 
                 mirror=True, 
                 showgrid=False)

fig['layout']['xaxis'].update(title='Date')

fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                   'paper_bgcolor': 'rgba(0,0,0,0)'},
                  font_color='black',
                  bargap=0,
                  yaxis=dict(title_text='Sales share (%)', 
                             side='right'
                             ),
                  yaxis2=dict(title_text='Price index',
                              side='left'))

fig.show()
# fig.write_image('./compare_art.pdf')